<a href="https://colab.research.google.com/github/dtabuena/Images/blob/main/Cell_Pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-bioformats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import javabridge
import bioformats
import skimage
from skimage import filters
from matplotlib import pyplot as plt
import matplotlib
import os
import scipy
from scipy import ndimage as ndi
javabridge.start_vm(class_path=bioformats.JARS)



def get_drobox_folder(link, new_filename):
    'Download a folder from dropbox and unzip'
    zipped_file_path = "/content/"+new_filename + ".zip"
    unzipped_file_path = "/content/"+new_filename
    if not( os.path.exists(zipped_file_path)):
        !wget -O $zipped_file_path $link    # download with new name
    !echo A | unzip $zipped_file_path -d $unzipped_file_path 
    return new_filename

def get_sub_files(rootdir):
    'Recursively search subfolders and return a list of all files'
    file_list =[]
    for rootdir, dirs, files in os.walk(rootdir): 
            file_list.extend([os.path.join(rootdir,f) for f in files])
    return file_list

folder = get_drobox_folder('https://www.dropbox.com/sh/rv7944fsk9p1mvk/AAC0E8ZTPkr-pfgK8zdC730Sa?dl=0', 'images')
file_list = get_sub_files(folder)
file_list.sort()

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Archive:  /content/images.zip
mapname:  conversion of  failed
replace /content/images/MDGA2_08-24-22_DG_40X_2XZoom_1.oir? [y]es, [n]o, [A]ll, [N]one, [r]ename:  extracting: /content/images/MDGA2_08-24-22_DG_40X_2XZoom_1.oir  
 extracting: /content/images/MDGA2_08-25-22_DG_40X_2XZoom_1.oir  
 extracting: /content/images/Nell2_08-04-22_DG_40X_2XZoom_1.oir  
 extracting: /content/images/Nell2_08-04-22_CA3_40X_2XZoom_1.oir  
 extracting: /content/images/RNF182_08-15-22_CA3_40X_2XZoom_1.oir  
 extracting: /content/images/MDGA2_08-22-22_S001_DG_40X_2XZoom_1.oir  
 extracting: /content/images/Nell2_08-10-22__S001_DG_40X_2XZoom_2.oir  
 extracting: /content/images/RNF182_08-17-22_S001_DG_40X_2XZoom_1.oir  
 extracting: /content/images/Nell2_08-10-22__S001_CA3_40X_2XZoom_2.oir  
 extracting: /content/images/RNF182_08-12-22_S001_CA3_40X_2XZoom_1.oir  
 extracting: /content/images/Other/DifferentExposure_RNF182_08-15-22_DG_40X_2XZoom_1.oir  
 extracting: /content/images/Other/BrighterSettings_Nel

In [ ]:
my_reader = bioformats.get_image_reader('my_image', file_list[1])

c_dim = my_reader.rdr.getSizeC()
z_dim = my_reader.rdr.getSizeZ()
x_dim = my_reader.rdr.getSizeX()
y_dim = my_reader.rdr.getSizeY()
print(x_dim,y_dim,c_dim,z_dim)

image = np.empty([x_dim,y_dim,c_dim,z_dim])
for zi in range(z_dim):
    img_zi = my_reader.read(c=None, z=zi,rescale=True)
    image[:,:,:,zi] = img_zi


image_zproj = np.max(image,axis=3)
# image = np.expand_dims(image_zproj, axis=-1)
print(image.shape)
for ci in range(image.shape[2]):
    cim = image[:,:,ci,:]
    flat = cim.flatten()
    max= np.percentile(flat,100)
    assert max>0 , print(flat)
    cim = cim/max
    image[:,:,ci,:] = cim

800 800 3 16
(800, 800, 3, 16)


In [ ]:
import os
os.environ["CELLPOSE_LOCAL_MODELS_PATH"] = "/PATH_FOR_MODELS/"

!pip install cellpose
from cellpose import models

# model_type='cyto' or 'nuclei' or 'cyto2'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model = models.Cellpose(model_type='cyto')
# imgs = [image[:,:,2,zi] for zi in range(image.shape[3])]
# print(imgs)
masks, flows, styles, diams = model.eval(image, diameter=15, channels=[0,0],do_3D=True)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# # from cellpose import models
# # from cellpose.io import imread

# # # model_type='cyto' or 'nuclei' or 'cyto2'
# # model = models.Cellpose(model_type='cyto')

# # list of files
# # PUT PATH TO YOUR FILES HERE!
# files = ['/media/carsen/DATA1/TIFFS/onechan.tif']

# imgs = [imread(f) for f in files]
# nimg = len(imgs)

# # define CHANNELS to run segementation on
# # grayscale=0, R=1, G=2, B=3
# # channels = [cytoplasm, nucleus]
# # if NUCLEUS channel does not exist, set the second channel to 0
# channels = [[0,0]]
# # IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# # channels = [0,0] # IF YOU HAVE GRAYSCALE
# # channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# # channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# # if diameter is set to None, the size of the cells is estimated on a per image basis
# # you can set the average cell `diameter` in pixels yourself (recommended)
# # diameter can be a list or a single number for all images

# masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)